# Frac Production Data Cleaning - Oklahoma

In [1]:
# Necessary imports
import numpy as np
import pandas as pd
# Cleaning import
import re

import warnings
warnings.filterwarnings('ignore')

## Oklahoma Production Data


## Engineer Well Data
Let's try this data set to see what it has.

In [2]:
df = pd.read_excel('OK/DI Engineering Explorer well roll up.xlsx')
df.head()

,abstract,allocFlag,api10,API12,azimuth,basin,bottomHoleTemp,bottomHoleTempDepth,casingPressure,chokeSize,...,wellNumber,wellType,peakProdDate,peak3monCumulativeGas,peak3monCumulativeOil,peak6monCumulativeGas,peak6monCumulativeOil,peak12monCumulativeGas,peak12monCumulativeOil,tvdSource
0,NaN,NaN,4237530074,423753007400,NaN,NaN,NaN,NaN,NaN,NaN,...,57R,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,X,3505123830,350512383000,1.0,ANADARKO BASIN,NaN,NaN,NaN,64/64,...,3-10-3XH,OIL,2015-02-01,60997.0,29497.0,130487.0,43864.0,243861.0,58129.0,Directional Survey
2,NaN,NaN,4229580553,422958055300,NaN,NaN,NaN,NaN,NaN,NaN,...,1,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,X,3501924811,350192481100,NaN,SOUTH OKLAHOMA FOLDED BELT,NaN,NaN,NaN,48/64,...,2018-01-01 00:00:00,GAS,2003-06-01,20679.0,0.0,24289.0,0.0,29161.0,0.0,Calculated
4,2114,X,4221133163,422113316300,NaN,ANADARKO BASIN,228.0,11781.0,NaN,1,...,7,GAS,2005-03-01,71859.0,741.0,110851.0,799.0,160752.0,825.0,Calculated


Since there are too many columns to look at at once, print the column name and contents for one well to determine what information is in the data frame.

In [3]:
for col in df.columns:
    print(col, '\t\t', df.loc[6,col])

abstract 		 nan
allocFlag 		 X
api10 		 3504523268
API12 		 350452326800
azimuth 		 356.0
basin 		 ANADARKO BASIN
bottomHoleTemp 		 nan
bottomHoleTempDepth 		 nan
casingPressure 		 nan
chokeSize 		 OPEN
completionDate 		 2012-12-20 00:00:00
county 		 ELLIS (OK)
cum3MonthsBoe 		 11096.0
cum3MonthsGas 		 14365.0
cum3MonthsOil 		 8702.0
cum3MonthsWater 		 0.0
cum6MonthsBoe 		 16487.0
cum6MonthsGas 		 16363.0
cum6MonthsOil 		 13760.0
cum6MonthsWater 		 0.0
cum12MonthsBoe 		 20926.0
cum12MonthsGas 		 15813.0
cum12MonthsOil 		 18291.0
cum12MonthsWater 		 0.0
cum24MonthsBoe 		 24710.0
cum24MonthsGas 		 17774.0
cum24MonthsOil 		 21748.0
cum24MonthsWater 		 0.0
cum60MonthsBoe 		 nan
cum60MonthsGas 		 nan
cum60MonthsOil 		 nan
cum60MonthsWater 		 nan
cumTotalGas 		 26416.0
cumTotalOil 		 25211.0
cumTotalWater 		 0.0
cumulativeGasOilRatio 		 1048.0
EURGas 		 26416.0
EUROil 		 37746.0
field 		 HIGGINS SOUTH
first3MonthsGasOilRatio 		 1651.0
firstGasOilRatio 		 1831.0
firstProductionDate 		 2013-01

Columns of interest:
- api12 is the API number - I will need to __add two 0's at the end to make it a 14-digit API number, like the rest have been__
- lateralLength is the horizontal length - I will need to __rename__
- perfInterval - I need to __rename__
- proppantLbsPerFoot __this could be a great feature to engineer for the Colorado wells__
- totalDepth 
- totalProppantLbs is the sand mass - I might not need the fracfocus data
- treatmentType might be the type of frac method used - I again might not need the fracfocus data, but I should __check the numbers of each type.__
- cum6MonthsBoe is my first 6 month's production - I will need to __re-name this to match the others for modeling__

In [4]:
# Fixing the length of the API numbers and setting them as the index
okwells = df[['API12','lateralLength','perfInterval','proppantLbsPerFoot',
             'totalDepth','totalProppantLbs','treatmentType','cum6MonthsBoe']]
# Renaming columns
okwells = okwells.rename(columns={'API12':'APINumber',
                                  'lateralLength':'HorizontalLength',
                                  'perfInterval':'GrossPerforatedInterval',
                                  'proppantLbsPerFoot':'SandMassPerFoot',
                                  'totalDepth':'totalDepth',
                                  'totalProppantLbs':'sandmass',
                                  'treatmentType':'treatmentType',
                                  'cum6MonthsBoe':'First6BOE'})
okwells.index = okwells.APINumber.map(lambda x: x*100)
okwells.head()

,APINumber,HorizontalLength,GrossPerforatedInterval,SandMassPerFoot,totalDepth,sandmass,treatmentType,First6BOE
APINumber,,,,,,,,
42375300740000,423753007400,NaN,NaN,NaN,1795.0,45000.0,UNKNOWN,NaN
35051238300000,350512383000,5575.0,5307.0,637.0,17821.0,3382619.0,SLICKWATER (HC),62733.0
42295805530000,422958055300,NaN,NaN,NaN,10699.0,212000.0,UNKNOWN,NaN
35019248110000,350192481100,NaN,10.0,NaN,4300.0,NaN,NaN,4048.0
42211331630000,422113316300,249.0,1333.0,427.0,12616.0,568580.0,UNKNOWN,19274.0


Fantastic. Looks like there's a fair amount of missing data.  Let's check that out first.

In [5]:
okwells.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67934 entries, 42375300740000 to 35049245950000
Data columns (total 8 columns):
APINumber                  67934 non-null int64
HorizontalLength           20943 non-null float64
GrossPerforatedInterval    46559 non-null float64
SandMassPerFoot            26274 non-null float64
totalDepth                 65252 non-null float64
sandmass                   35019 non-null float64
treatmentType              45921 non-null object
First6BOE                  49890 non-null float64
dtypes: float64(6), int64(1), object(1)
memory usage: 4.7+ MB


Wow! Oklahoma has a lot of wells! We have complete data for around 26K wells, and for the data that we also had for Colorado (meaning not including proppant pounds per foot), we have 35K wells.  This is fantastic.  Let's drop all rows with NaN values and see what the shape of our data is.

In [6]:
okwells2 = okwells.dropna(axis=0, how='any')
okwells2.shape

(14275, 8)

Alright, after dropping all Na values, I have 14K wells, but that's still more than I had for Colorado.

### Treatment Type
Now I want to check out that treatment type to see what's in it.

In [7]:
okwells2.treatmentType.value_counts()

SLICKWATER (HC)     6895
UNKNOWN             4842
SW-GEL HYBRID        921
LINEAR GEL (HC)      920
CROSSLINKED (HC)     541
LINEAR GEL            74
ACID                  33
SW-XL HYBRID          25
CROSSLINKED           14
ENERGIZED FLUIDS      10
Name: treatmentType, dtype: int64

This looks great - I will need to clean up some of these and combine them, but these will be able to replace the fracfocus data. I will also have to lose the almost 5K wells whose information I don't have, but I might be able to find that from the fracfocus data.

In [8]:
okwells2['slick'] = okwells2.treatmentType.map(lambda x: 1 if re.search('SLICK|SW', x) else 0)
okwells2['gel'] = okwells2.treatmentType.map(lambda x: 1 if re.search('LINEAR|SW-GEL', x) else 0)
okwells2['xlinkgel'] = okwells2.treatmentType.map(lambda x: 1 if re.search('XL|CROSS', x) else 0)
okwells2['hybrid'] = okwells2.treatmentType.map(lambda x: 1 if re.search('HYBRID', x) else 0)

In [9]:
okwells2.head()

,APINumber,HorizontalLength,GrossPerforatedInterval,SandMassPerFoot,totalDepth,sandmass,treatmentType,First6BOE,slick,gel,xlinkgel,hybrid
APINumber,,,,,,,,,,,,
35051238300000,350512383000,5575.0,5307.0,637.0,17821.0,3382619.0,SLICKWATER (HC),62733.0,1,0,0,0
42211331630000,422113316300,249.0,1333.0,427.0,12616.0,568580.0,UNKNOWN,19274.0,0,0,0,0
35045232680000,350452326800,4235.0,4044.0,254.0,12216.0,1026000.0,UNKNOWN,16487.0,0,0,0,0
42483338530000,424833385300,4617.0,4682.0,882.0,18104.0,4131188.0,SLICKWATER (HC),132965.0,1,0,0,0
42375318560000,423753185600,5482.0,5189.0,1507.0,14817.0,7817621.0,SLICKWATER (HC),155981.0,1,0,0,0


This all looks good.  

### Frac Method Supplement
Now I will load the frac method data in so I can fill in any information that is in that document but not in this well set.

In [10]:
# Reading in frac method data
fracmethod = pd.read_csv('fracmethod_clean.csv', index_col=0)
fracmethod.head()

,slick,gel,xlinkgel,hybrid,sandmass
APINumber,,,,,
4226932868,1,0,1,1.0,8552.338261
1007203590000,1,1,0,1.0,177493.467769
1007203600000,1,1,0,1.0,151188.848979
1007203610000,1,1,0,1.0,219070.601388
1007203620000,1,1,0,1.0,245962.571934


Now I will take the unknown treatment type wells and supplement them with what frac focus method data I have.

In [11]:
# Selecting all Unknown method wells
for i in okwells2[okwells2.treatmentType == 'UNKNOWN'].index:
    # If the API number is also in the frac focus index, use the frac method
    # data in the Oklahoma wells information
    if i in fracmethod.index:
        okwells2.loc[i, 'slick'] = fracmethod.loc[i, 'slick']
        okwells2.loc[i, 'gel'] = fracmethod.loc[i, 'gel']
        okwells2.loc[i, 'xlinkgel'] = fracmethod.loc[i, 'xlinkgel']
    # If the information isn't in either place, drop the well
    else:
        okwells2 = okwells2.drop(i, axis=0)

In [12]:
# Checking on the status of the unknown wells
okwells2[okwells2.treatmentType=='UNKNOWN'].head()

,APINumber,HorizontalLength,GrossPerforatedInterval,SandMassPerFoot,totalDepth,sandmass,treatmentType,First6BOE,slick,gel,xlinkgel,hybrid
APINumber,,,,,,,,,,,,
35019256750000,350192567500,4804.0,4706.0,839.0,18656.0,3946056.0,UNKNOWN,64519.0,0,0,0,0
35051236460000,350512364600,3906.0,3774.0,730.0,19341.0,2753863.0,UNKNOWN,126333.0,0,0,0,0
42211351390000,422113513900,5719.0,5379.0,366.0,14720.0,1970760.0,UNKNOWN,24225.0,0,0,0,0
42295338300000,422953383000,3984.0,4016.0,591.0,11866.0,2373360.0,UNKNOWN,30252.0,0,0,0,0
35007255660000,350072556600,4172.0,3969.0,121.0,11135.0,479040.0,UNKNOWN,13086.0,0,0,0,0


Great, we now have all of the information about frac methods in the data frame that we have on hand.

### Vertical Depth
Now, I need to calculate the vertical depth from the total depth and lateral length of the well.

In [13]:
okwells2['VerticalDepth'] = okwells2['totalDepth'] - okwells2['HorizontalLength']

In [14]:
okwells2.head()

,APINumber,HorizontalLength,GrossPerforatedInterval,SandMassPerFoot,totalDepth,sandmass,treatmentType,First6BOE,slick,gel,xlinkgel,hybrid,VerticalDepth
APINumber,,,,,,,,,,,,,
35051238300000,350512383000,5575.0,5307.0,637.0,17821.0,3382619.0,SLICKWATER (HC),62733.0,1,0,0,0,12246.0
42483338530000,424833385300,4617.0,4682.0,882.0,18104.0,4131188.0,SLICKWATER (HC),132965.0,1,0,0,0,13487.0
42375318560000,423753185600,5482.0,5189.0,1507.0,14817.0,7817621.0,SLICKWATER (HC),155981.0,1,0,0,0,9335.0
35043232790000,350432327900,4282.0,4194.0,722.0,13250.0,3029140.0,SLICKWATER (HC),10426.0,1,0,0,0,8968.0
35149215590000,351492155900,4664.0,4658.0,665.0,17399.0,3098539.0,SLICKWATER (HC),51461.0,1,0,0,0,12735.0


Fantastic!  Now I just have to engineer the same features as in the Colorado Data.

## Feature Engineering

Here I will engineer the binned horizontal length, the square root of non-perforated feet, and the square root of sand mass.

In [15]:
# Creating bins for the three general horizontal legnths
bins = [0,6500,8500,250000]
okwells2['hzlen_bin'] = pd.cut(okwells2['HorizontalLength'], bins,
                              labels = ['<1', '1-2', '>2'])

okwells2['nphf_sqrt'] = np.sqrt(okwells2.HorizontalLength - okwells2.GrossPerforatedInterval + 6000)

okwells2['sqrtsandmass'] = np.sqrt(okwells2.sandmass)
okwells2.head()

,APINumber,HorizontalLength,GrossPerforatedInterval,SandMassPerFoot,totalDepth,sandmass,treatmentType,First6BOE,slick,gel,xlinkgel,hybrid,VerticalDepth,hzlen_bin,nphf_sqrt,sqrtsandmass
APINumber,,,,,,,,,,,,,,,,
35051238300000,350512383000,5575.0,5307.0,637.0,17821.0,3382619.0,SLICKWATER (HC),62733.0,1,0,0,0,12246.0,<1,79.170702,1839.189767
42483338530000,424833385300,4617.0,4682.0,882.0,18104.0,4131188.0,SLICKWATER (HC),132965.0,1,0,0,0,13487.0,<1,77.038951,2032.532411
42375318560000,423753185600,5482.0,5189.0,1507.0,14817.0,7817621.0,SLICKWATER (HC),155981.0,1,0,0,0,9335.0,<1,79.328431,2796.000894
35043232790000,350432327900,4282.0,4194.0,722.0,13250.0,3029140.0,SLICKWATER (HC),10426.0,1,0,0,0,8968.0,<1,78.025637,1740.442472
35149215590000,351492155900,4664.0,4658.0,665.0,17399.0,3098539.0,SLICKWATER (HC),51461.0,1,0,0,0,12735.0,<1,77.498387,1760.266741


Now, I will select the best features from the Colorado data to use in testing the model

In [16]:
# Selecting the same features from the Colorado data
okfeatures = okwells2[['gel','slick','xlinkgel',
                      'VerticalDepth','hzlen_bin','nphf_sqrt',
                      'sqrtsandmass','First6BOE',
                      'SandMassPerFoot','hybrid']]
# Making dummy variables for the binned variables.
okfeatures = pd.get_dummies(okfeatures)

In [17]:
# Writing Oklahoma features to csv for modeling
okfeatures.to_csv('okfeatures.csv')